In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("features.csv")

# just made one head roll
df['head_roll_abs'] = np.abs(df['head_roll'])
features = [
    'brow_ratio',
    'smile_up',
    'mouth_open',
    'mouth_width',
    'head_roll_abs'
]
X = df[features]
y = df['label'].apply(lambda x: 1 if 'confused' in x.lower() else 0)

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
inigrid = {'max_depth': [2, 3, 4, 5]}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(
    DecisionTreeClassifier(random_state=42, class_weight='balanced'),
    inigrid,
    cv=cv,
    scoring='f1'
)
grid.fit(X, y)
print("Best:", grid.best_params_, "CV f1:", grid.best_score_)
clf = grid.best_estimator_


Best: {'max_depth': 5} CV f1: 0.78223483361033


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['not_confused', 'confused']))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

not_confused       0.90      0.92      0.91        50
    confused       0.79      0.75      0.77        20

    accuracy                           0.87        70
   macro avg       0.85      0.83      0.84        70
weighted avg       0.87      0.87      0.87        70

[[46  4]
 [ 5 15]]


In [8]:
from sklearn.tree import export_text

rules = export_text(clf, feature_names=list(X.columns))
print(rules)


|--- smile_up <= -0.23
|   |--- smile_up <= -0.24
|   |   |--- class: 1
|   |--- smile_up >  -0.24
|   |   |--- brow_ratio <= 0.22
|   |   |   |--- class: 0
|   |   |--- brow_ratio >  0.22
|   |   |   |--- class: 1
|--- smile_up >  -0.23
|   |--- mouth_open <= 0.02
|   |   |--- brow_ratio <= 0.21
|   |   |   |--- smile_up <= -0.20
|   |   |   |   |--- class: 0
|   |   |   |--- smile_up >  -0.20
|   |   |   |   |--- head_roll_abs <= 1.22
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- head_roll_abs >  1.22
|   |   |   |   |   |--- class: 0
|   |   |--- brow_ratio >  0.21
|   |   |   |--- head_roll_abs <= 3.02
|   |   |   |   |--- brow_ratio <= 0.22
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- brow_ratio >  0.22
|   |   |   |   |   |--- class: 0
|   |   |   |--- head_roll_abs >  3.02
|   |   |   |   |--- class: 0
|   |--- mouth_open >  0.02
|   |   |--- mouth_open <= 0.03
|   |   |   |--- head_roll_abs <= 1.06
|   |   |   |   |--- smile_up <= -0.20
|   |   |   |   |   |-

In [14]:
import os
os.makedirs("models", exist_ok=True)

In [13]:
import joblib
joblib.dump(clf, "models/confusion_tree.joblib")


['models/confusion_tree.joblib']